# Function Transformations

**Function transformation** is a way to describe a specific type of high order functions. 

> It is when a function takes a function(s) as **input** and returns a **new function**

For example:

In [3]:
def multiply(x, y):
    return x * y

def add(x, y):
    return x + y

# self_math is a higher order function
# input: a function that takes two arguments and returns a value
# output: a new function that takes one argument and returns a value
def self_math(math_func):
    def inner_func(x):
        return math_func(x, x)
    return inner_func

print(self_math(multiply)(5))
print(self_math(add)(5))

25
10


**`self_math`** takes a function that operates on two different parameters and return a function that operates on one parameter twice

---
In the following example, **`get_logger`** takes a function that operates over two strings as input, and return a function that makes the changes over those strings

In [4]:
def get_logger(formatter):
    def logger(first_str, second_str):
        print(formatter(first_str, second_str))
    return logger

In [5]:
def test(first, errors, formatter):
    print("Logs:")
    logger = get_logger(formatter)
    for err in errors:
        logger(first, err)
    print("====================================")


def colon_delimit(first, second):
    return f"{first}: {second}"


def dash_delimit(first, second):
    return f"{first} - {second}"


def main():
    db_errors = [
        "out of memory",
        "cpu is pegged",
        "networking issue",
        "invalid syntax",
    ]
    test("Doc2Doc FATAL", db_errors, colon_delimit)

    mail_errors = [
        "email too large",
        "non alphanumeric symbols found",
    ]
    test("Doc2Doc WARNING", mail_errors, dash_delimit)


main()

Logs:
Doc2Doc FATAL: out of memory
Doc2Doc FATAL: cpu is pegged
Doc2Doc FATAL: networking issue
Doc2Doc FATAL: invalid syntax
Logs:
Doc2Doc WARNING - email too large
Doc2Doc WARNING - non alphanumeric symbols found


## Example

This exercise is about a function that takes another **function** and a **list** of valid formats as param. It **returns** a new **function** that takes two param: 

- `name` of the file to be converted
- `content` of the file to be converted

Only if the extension of filename is in valid_formats list, then it should return the result of calling conversion_function on the content

In [26]:
def doc_format_checker_and_converter(conversion_function, valid_formats):
    
    def inner_funct(filename, content):
        if filename.split('.')[1] in valid_formats:
            return conversion_function(content)
        else:
            raise ValueError("invalid file format")
    return inner_funct
    
#These are the conversion_functions
def capitalize_content(content):
    return content.upper()

def reverse_content(content):
    return content[::-1]

In [27]:
run_cases = [
    (
        capitalize_content,
        "sample.txt",
        "I really don't feel like screaming today.",
        ["txt", "md", "doc"],
        "I REALLY DON'T FEEL LIKE SCREAMING TODAY.",
    ),
    (
        reverse_content,
        "testing.doc",
        "This is probably how they write in the red room in Twin Peaks...",
        ["txt", "md", "doc"],
        "...skaeP niwT ni moor der eht ni etirw yeht woh ylbaborp si sihT",
    ),
]

submit_cases = run_cases + [
    (
        capitalize_content,
        "test.docx",
        "Okay actually I do feel like screaming today.",
        ["txt", "md", "doc"],
        "invalid file format",
    ),
    (
        reverse_content,
        "end.ppt",
        "Cherry pie and coffee anyone?",
        ["txt", "md", "doc"],
        "invalid file format",
    ),
    (
        capitalize_content,
        "sample.doc",
        "I really do feel like eating today.",
        ["txt", "md", "doc"],
        "I REALLY DO FEEL LIKE EATING TODAY.",
    ),
    (
        reverse_content,
        "testing.md",
        "The owls are not what they seem.",
        ["txt", "md", "doc"],
        ".mees yeht tahw ton era slwo ehT",
    ),
]


def test(conversion_func, filename, doc_content, valid_formats, expected_output):
    print("---------------------------------")
    print(f"Inputs:")
    print(f" * conversion_func: {conversion_func.__name__}")
    print(f" * filename: {filename}")
    print(f" * doc_content: {doc_content}")
    print(f" * valid_formats: {valid_formats}")
    print(f"Expected: {expected_output}")
    try:
        result = doc_format_checker_and_converter(conversion_func, valid_formats)(
            filename, doc_content
        )
    except Exception as e:
        if not isinstance(e, ValueError):
            return False
        result = str(e)
    print(f"Actual:   {result}")
    if result == expected_output:
        print("Pass")
        return True
    print("Fail")
    return False


def main():
    passed = 0
    failed = 0
    skipped = len(submit_cases) - len(test_cases)
    for test_case in test_cases:
        correct = test(*test_case)
        if correct:
            passed += 1
        else:
            failed += 1
    if failed == 0:
        print("============= PASS ==============")
    else:
        print("============= FAIL ==============")
    if skipped > 0:
        print(f"{passed} passed, {failed} failed, {skipped} skipped")
    else:
        print(f"{passed} passed, {failed} failed")


test_cases = submit_cases
if "__RUN__" in globals():
    test_cases = run_cases

main()

---------------------------------
Inputs:
 * conversion_func: capitalize_content
 * filename: sample.txt
 * doc_content: I really don't feel like screaming today.
 * valid_formats: ['txt', 'md', 'doc']
Expected: I REALLY DON'T FEEL LIKE SCREAMING TODAY.
Actual:   I REALLY DON'T FEEL LIKE SCREAMING TODAY.
Pass
---------------------------------
Inputs:
 * conversion_func: reverse_content
 * filename: testing.doc
 * doc_content: This is probably how they write in the red room in Twin Peaks...
 * valid_formats: ['txt', 'md', 'doc']
Expected: ...skaeP niwT ni moor der eht ni etirw yeht woh ylbaborp si sihT
Actual:   ...skaeP niwT ni moor der eht ni etirw yeht woh ylbaborp si sihT
Pass
---------------------------------
Inputs:
 * conversion_func: capitalize_content
 * filename: test.docx
 * doc_content: Okay actually I do feel like screaming today.
 * valid_formats: ['txt', 'md', 'doc']
Expected: invalid file format
Actual:   invalid file format
Pass
---------------------------------
Inputs:

## Example

The **`get_filter_cmd`** takes two functions as input and returns the function **`filter_cmd`**. This one take two str as inputs: content and option

In [28]:
def get_filter_cmd(filter_one, filter_two):
    def filter_cmd(content, option='--one'):
        '''
        this function filters and returns the content according to the 
        input option: if one, use filter_one; if two, use filter_two; if three
        use filter_one first and then use filter_two
        '''
        if option == '--one':
            return filter_one(content)
        elif option == '--two':
            return filter_two(content)
        elif option == '--three':
            return filter_two(filter_one(content))
        else:
            raise Exception('invalid option')
    return filter_cmd

def replace_bad(text):
    return text.replace("bad", "good")
def replace_ellipsis(text):
    return text.replace("..", "...")
def fix_ellipsis(text):
    return text.replace("....", "...")

In [29]:
run_cases = [
    (
        replace_bad,
        replace_ellipsis,
        [
            (
                (
                    "I'm bad, and that's good. I will never be good, and that's not bad..",
                ),
                "I'm good, and that's good. I will never be good, and that's not good..",
            ),
            (
                (
                    "I'm bad, and that's good. I will never be good, and that's not bad..",
                    "--one",
                ),
                "I'm good, and that's good. I will never be good, and that's not good..",
            ),
            (
                (
                    "I'm bad, and that's good. I will never be good, and that's not bad..",
                    "--two",
                ),
                "I'm bad, and that's good. I will never be good, and that's not bad...",
            ),
            (
                (
                    "I'm bad, and that's good. I will never be good, and that's not bad..",
                    "--three",
                ),
                "I'm good, and that's good. I will never be good, and that's not good...",
            ),
        ],
    ),
]

submit_cases = run_cases + [
    (
        replace_ellipsis,
        fix_ellipsis,
        [
            (
                (
                    "There's no place like home.. but sometimes, it's nice to get away... and explore....",
                ),
                "There's no place like home... but sometimes, it's nice to get away.... and explore......",
            ),
            (
                (
                    "There's no place like home.. but sometimes, it's nice to get away... and explore....",
                    "--one",
                ),
                "There's no place like home... but sometimes, it's nice to get away.... and explore......",
            ),
            (
                (
                    "There's no place like home.. but sometimes, it's nice to get away... and explore....",
                    "--two",
                ),
                "There's no place like home.. but sometimes, it's nice to get away... and explore...",
            ),
            (
                (
                    "There's no place like home.. but sometimes, it's nice to get away... and explore....",
                    "--three",
                ),
                "There's no place like home... but sometimes, it's nice to get away... and explore.....",
            ),
            (
                (
                    "There's no place like home.. but sometimes, it's nice to get away... and explore....",
                    "",
                ),
                "invalid option",
            ),
        ],
    ),
]


def test(filter_one, filter_two, test_cases):
    print("---------------------------------")
    print(f"Input functions: {filter_one.__name__} and {filter_two.__name__}")
    filter_cmd = get_filter_cmd(filter_one, filter_two)
    failed = False
    for case in test_cases:
        try:
            result = filter_cmd(*case[0])
        except Exception as e:
            result = str(e)
        expected_output = case[1]
        print(f"Expected: {expected_output}")
        print(f"Actual:   {result}")
        if result != expected_output:
            failed = True
            print("Fail")
        else:
            print("Pass")
    passed = not failed
    return passed


def main():
    passed = 0
    failed = 0
    skipped = len(submit_cases) - len(test_cases)
    for test_case in test_cases:
        correct = test(*test_case)
        if correct:
            passed += 1
        else:
            failed += 1
    if failed == 0:
        print("============= PASS ==============")
    else:
        print("============= FAIL ==============")
    if skipped > 0:
        print(f"{passed} passed, {failed} failed, {skipped} skipped")
    else:
        print(f"{passed} passed, {failed} failed")


test_cases = submit_cases
if "__RUN__" in globals():
    test_cases = run_cases

main()

---------------------------------
Input functions: replace_bad and replace_ellipsis
Expected: I'm good, and that's good. I will never be good, and that's not good..
Actual:   I'm good, and that's good. I will never be good, and that's not good..
Pass
Expected: I'm good, and that's good. I will never be good, and that's not good..
Actual:   I'm good, and that's good. I will never be good, and that's not good..
Pass
Expected: I'm bad, and that's good. I will never be good, and that's not bad...
Actual:   I'm bad, and that's good. I will never be good, and that's not bad...
Pass
Expected: I'm good, and that's good. I will never be good, and that's not good...
Actual:   I'm good, and that's good. I will never be good, and that's not good...
Pass
---------------------------------
Input functions: replace_ellipsis and fix_ellipsis
Expected: There's no place like home... but sometimes, it's nice to get away.... and explore......
Actual:   There's no place like home... but sometimes, it's nice